In [5]:
# 디버깅용 스크립트
#
# focc_minutes.py가 0개를 수집하는 문제를 해결하기 위해,
# 캘린더 페이지의 현재 HTML 구조를 파일로 저장합니다.

import requests
import os

# --- 설정 ---
CALENDAR_URL = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
# 저장할 경로
DOWNLOADS_DIR = os.path.join(os.path.expanduser('~'), 'Downloads', 'statements')
DEBUG_FILE = os.path.join(DOWNLOADS_DIR, "debug_calendar.html")

print(f"디버깅용 HTML 파일 수집 시작...")
print(f"URL: {CALENDAR_URL}")

try:
    # 폴더 생성 확인
    if not os.path.exists(DOWNLOADS_DIR):
        os.makedirs(DOWNLOADS_DIR)
    
    # --- HTML 요청 ---
    response = requests.get(CALENDAR_URL, headers=HEADERS)
    response.raise_for_status() # 오류가 있으면 예외 발생
    
    # --- 파일로 저장 ---
    with open(DEBUG_FILE, 'w', encoding='utf-8') as f:
        f.write(response.text)
        
    print(f"\n--- 성공 ---")
    print(f"HTML 내용을 '{DEBUG_FILE}' 파일로 저장했습니다.")
    print("이 파일을 열어서 내용을 확인하거나, 저에게 다시 공유해주세요.")

except requests.exceptions.RequestException as e:
    print(f"\n--- 오류 ---")
    print(f"URL에 접근하는 중 오류가 발생했습니다: {e}")
except IOError as e:
    print(f"\n--- 오류 ---")
    print(f"파일을 저장하는 중 오류가 발생했습니다: {e}")
except Exception as e:
    print(f"\n--- 알 수 없는 오류 ---")
    print(e)

디버깅용 HTML 파일 수집 시작...
URL: https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm

--- 성공 ---
HTML 내용을 'C:\Users\82109\Downloads\statements\debug_calendar.html' 파일로 저장했습니다.
이 파일을 열어서 내용을 확인하거나, 저에게 다시 공유해주세요.


In [7]:
# FOMC 'Minutes' (의사록) 전용 크롤러 (2020 ~ 현재)
#
# [2025-10-30 v2 수정]
# - 날짜 파싱(parse_meeting_date) 오류를 전면 수정했습니다.
# - 메인 루프에서 '월'과 '일'을 정확히 찾아 함수로 전달합니다.
#
# 참고: 이 캘린더 페이지는 2020년 이후의 데이터만 포함하고 있습니다.
#
# [중요] 실행 전 설정:
# 1. 터미널에서 `pip install requests beautifulsoup4 PyMuPDF tqdm`을 실행하여
#    라이브러리를 설치하세요.

import requests
from bs4 import BeautifulSoup
import fitz  # PyMuPDF
import json
import time
import os
import datetime
from urllib.parse import urljoin
from tqdm import tqdm
import re

# --- 1. 설정 (Configuration) ---
BASE_URL = "https://www.federalreserve.gov"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
CALENDAR_URL = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"
DOWNLOADS_DIR = os.path.join(os.path.expanduser('~'), 'Downloads', 'statements')
OUTPUT_FILE = os.path.join(DOWNLOADS_DIR, "fomc_minutes_2020-present.jsonl") 
START_YEAR = 2020
END_YEAR = datetime.datetime.now().year

MONTH_MAP = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12,
    'Jan/Feb': 1, 'Apr/May': 4, 'Oct/Nov': 10
}

# --- 2. 헬퍼 함수 (Helper Functions) ---

def save_to_jsonl(data, outfile):
    """데이터를 JSON Lines (.jsonl) 파일에 추가합니다."""
    try:
        output_dir = os.path.dirname(outfile)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        with open(outfile, 'a', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False)
            f.write('\n')
    except IOError as e:
        print(f"  [오류] 파일 쓰기 실패: {e}")

def get_soup(url):
    """지정된 URL의 BeautifulSoup 객체를 반환합니다."""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"  [오류] URL에 접근할 수 없습니다: {url} ({e})")
        return None

def clean_text(text):
    """불필요한 공백과 줄바꿈 문자를 정제합니다."""
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def extract_text_from_link(link_url):
    """HTML 또는 PDF 링크에서 텍스트를 깨끗하게 추출합니다."""
    text = ""
    try:
        full_url = urljoin(BASE_URL, link_url)
        response = requests.get(full_url, headers=HEADERS, timeout=10)
        response.raise_for_status()

        if ".pdf" in full_url.lower():
            with fitz.open(stream=response.content, filetype="pdf") as doc:
                for page in doc:
                    text += page.get_text() + "\n"
        else:
            soup = BeautifulSoup(response.text, 'html.parser')
            # Fed 웹사이트의 본문 내용 영역
            content_div = soup.find('div', class_='col-xs-12 col-sm-8 col-md-8')
            if content_div:
                text = content_div.get_text(separator="\n", strip=True)
            else:
                text = soup.get_text(separator="\n", strip=True)

    except Exception as e:
        print(f"  [오류] 링크에서 텍스트 추출 실패: {full_url} ({e})")
        
    time.sleep(0.5) 
    return clean_text(text)

# [수정됨] 날짜 파싱 함수 (매개변수 및 로직 수정)
def parse_meeting_date(month_text, day_text, year):
    """ "January", "28-29", 2025를 datetime 객체로 변환 (회의 종료일 기준) """
    try:
        month_str = month_text.strip()
        
        # "22 (notation vote)" -> "22"
        # "16-17*" -> "16-17"
        day_search_text = day_text.strip().split(' ')[0].replace('*', '')
        
        month = MONTH_MAP.get(month_str)
        if not month: # "Apr/May" 같은 복합 월 처리
            month_str_first = month_str.split('/')[0]
            month = MONTH_MAP.get(month_str_first)
            
        # "28-29" -> 29, "30-1" -> 1, "3" -> 3
        day = int(re.split(r'[-/]', day_search_text)[-1]) 
        
        # 월이 넘어가는 "30-1" 같은 케이스 처리 (Apr/May)
        if month_str == 'Apr/May' and day == 1:
            month = 5 # 5월 1일
        elif month_str == 'Jan/Feb' and day == 1:
            month = 2 # 2월 1일
        elif month_str == 'Oct/Nov' and day == 1:
            month = 11 # 11월 1일

        return datetime.datetime(year, month, day)
    except Exception as e:
         # print(f"  [경고] 날짜 파싱 실패: '{month_str}', '{day_text}' ({year}년) - {e}")
         return None

# --- 3. 메인 스크래핑 함수 (수정됨) ---
def scrape_all_minutes(output_file):
    print(f"--- FOMC 'Minutes' (의사록) 스크래핑 시작 (2020년 이후) ---")
    print(f"대상 기간: {START_YEAR}년 ~ {END_YEAR}년")
    print(f"데이터 저장 위치: {output_file}\n")
    
    soup = get_soup(CALENDAR_URL)
    if not soup:
        print("[치명적 오류] 캘린더 페이지에 접근할 수 없습니다. 스크립트를 종료합니다.")
        return

    year_panels = soup.find_all('div', class_='panel-default')
    scraped_count = 0
    total_meetings = 0
    
    if not year_panels:
        print("[치명적 오류] 'panel-default' 클래스를 찾지 못했습니다. HTML 구조가 다시 변경된 것 같습니다.")
        return

    for year_panel in year_panels:
        h4 = year_panel.find('h4')
        if not h4 or not h4.text:
            continue
        
        year_match = re.search(r'(\d{4})', h4.text)
        if not year_match:
            continue
            
        year = int(year_match.group(1))
            
        if not (START_YEAR <= year <= END_YEAR):
            continue
            
        print(f"--- {year}년 'Minutes' 데이터 처리 중 ---")
        
        meetings = year_panel.find_all('div', class_='fomc-meeting')
        
        for meeting in tqdm(meetings, desc=f"{year}년 회의"):
            total_meetings += 1 # 총 회의 수 카운트
            
            # [수정됨] '월'과 '일'을 별도로 찾음
            month_element = meeting.find('div', class_='fomc-meeting__month')
            date_element = meeting.find('div', class_='fomc-meeting__date')
            
            # '월' 또는 '일' 정보가 없으면 이 회의는 건너뜀
            if not month_element or not date_element:
                # print(f"  [정보] {year}년 - '월' 또는 '일' 정보가 없는 항목(예: 각주) 건너뜀")
                continue
                
            month_text = month_element.get_text(strip=True)
            date_text = date_element.get_text(strip=True)

            # [수정됨] 날짜 파싱
            meeting_date = parse_meeting_date(month_text, date_text, year)
            
            # 날짜 파싱에 실패하면 건너뜀
            if meeting_date is None:
                print(f"  [경고] {year}년 '{month_text} {date_text}' - 날짜 파싱 실패. 건너뜁니다.")
                continue 

            minutes_div = meeting.find('div', class_='fomc-meeting__minutes')
            minutes_link = None
            if minutes_div:
                minutes_link = minutes_div.find('a') 

            if minutes_link:
                link_url = minutes_link.get('href')
                
                if not link_url.startswith('http') and not link_url.startswith('/'):
                    link_url = f"/monetarypolicy/fomc/minutes/{link_url}"

                source_type = "FOMC Minutes"
                text = extract_text_from_link(link_url)
                
                if text:
                    data = {
                        "date": meeting_date.strftime('%Y-%m-%d'), 
                        "year": year,
                        "source_type": source_type,
                        "url": urljoin(BASE_URL, link_url),
                        "text": text
                    }
                    save_to_jsonl(data, output_file)
                    scraped_count += 1
                else:
                    print(f"  [경고] {year} ({date_text}) - 텍스트 추출 실패 (링크: {link_url})")
            # else:
                # (주석 처리) 'Minutes' 링크가 없는 회의(예: 10월 28-29일)는 조용히 건너뜀
                # print(f"  > {year} ({month_text} {date_text}) - 'Minutes' 링크 없음 (예정된 회의 등)")
                        
    print(f"\n--- 스크래핑 완료 (2020년 이후) ---")
    print(f"총 {total_meetings}개의 회의 항목 중 {scraped_count}개의 'Minutes' 문서를 수집했습니다.")

# --- 4. 메인 실행 ---
if __name__ == "__main__":
    if not os.path.exists(DOWNLOADS_DIR):
        os.makedirs(DOWNLOADS_DIR)
        print(f"'{DOWNLOADS_DIR}' 폴더를 생성했습니다.")

    if os.path.exists(OUTPUT_FILE):
        os.remove(OUTPUT_FILE)
        print(f"기존 파일 '{OUTPUT_FILE}'을 삭제했습니다. 새로 수집합니다.")
    
    scrape_all_minutes(OUTPUT_FILE)
    
    print(f"\n--- 모든 'Minutes' 작업 완료 ---")
    print(f"최종 데이터가 '{OUTPUT_FILE}' 파일에 저장되었습니다.")


--- FOMC 'Minutes' (의사록) 스크래핑 시작 (2020년 이후) ---
대상 기간: 2020년 ~ 2025년
데이터 저장 위치: C:\Users\82109\Downloads\statements\fomc_minutes_2020-present.jsonl

--- 2025년 'Minutes' 데이터 처리 중 ---


2025년 회의: 100%|██████████| 9/9 [00:10<00:00,  1.13s/it]


--- 2024년 'Minutes' 데이터 처리 중 ---


2024년 회의: 100%|██████████| 8/8 [00:12<00:00,  1.52s/it]


--- 2023년 'Minutes' 데이터 처리 중 ---


...

--- 2021년 'Minutes' 데이터 처리 중 ---


2021년 회의: 100%|██████████| 8/8 [00:11<00:00,  1.49s/it]


--- 2020년 'Minutes' 데이터 처리 중 ---


2020년 회의: 100%|██████████| 14/14 [00:16<00:00,  1.17s/it]


--- 스크래핑 완료 (2020년 이후) ---
총 55개의 회의 항목 중 46개의 'Minutes' 문서를 수집했습니다.

--- 모든 'Minutes' 작업 완료 ---
최종 데이터가 'C:\Users\82109\Downloads\statements\fomc_minutes_2020-present.jsonl' 파일에 저장되었습니다.


In [10]:
# FOMC 'Minutes' (의사록) 전용 크롤러 (2000 ~ 2019년 과거 데이터)
#
# [2025-10-30 v3 수정]
# - 날짜 파싱 로직을 (MONTH_MAP) 기준으로 강화
# - "pages...", "Part 1" 등 불필요한 행을 건너뛰도록 수정
# - [핵심] re.match() (처음부터) -> re.search() (어디서든)로 변경
#
# 참고: 이 스크립트는 2000년부터 2019년까지의 데이터를 수집합니다.
#
# [중요] 실행 전 설정:
# 1. 터미널에서 `pip install requests beautifulsoup4 PyMuPDF tqdm`을 실행하여
#    라이브러리를 설치하세요.

import requests
from bs4 import BeautifulSoup
import fitz  # PyMuPDF
import json
import time
import os
import datetime
from urllib.parse import urljoin
from tqdm import tqdm
import re

# --- 1. 설정 (Configuration) ---
BASE_URL = "https://www.federalreserve.gov"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
HISTORICAL_URL_TEMPLATE = "https://www.federalreserve.gov/monetarypolicy/fomchistorical{year}.htm"
DOWNLOADS_DIR = os.path.join(os.path.expanduser('~'), 'Downloads', 'statements')
OUTPUT_FILE = os.path.join(DOWNLOADS_DIR, "fomc_minutes_2000-2019.jsonl")
START_YEAR = 2000 
END_YEAR = 2019

MONTH_MAP = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12,
    'Jan.': 1, 'Feb.': 2, 'Mar.': 3, 'Apr.': 4, 'Jun.': 6, 'Jul.': 7, 'Aug.': 8, 'Sept.': 9, 'Oct.': 10, 'Nov.': 11, 'Dec.': 12
}

# --- 2. 헬퍼 함수 (Helper Functions) ---

def save_to_jsonl(data, outfile):
    """데이터를 JSON Lines (.jsonl) 파일에 추가합니다."""
    try:
        output_dir = os.path.dirname(outfile)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        with open(outfile, 'a', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False)
            f.write('\n')
    except IOError as e:
        print(f"  [오류] 파일 쓰기 실패: {e}")

def get_soup(url):
    """지정된 URL의 BeautifulSoup 객체를 반환합니다."""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        response.encoding = response.apparent_encoding 
        return BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"  [오류] URL에 접근할 수 없습니다: {url} ({e})")
        return None

def clean_text(text):
    """불필요한 공백과 줄바꿈 문자를 정제합니다."""
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def extract_text_from_link(link_url):
    """HTML 또는 PDF 링크에서 텍스트를 깨끗하게 추출합니다."""
    text = ""
    try:
        full_url = urljoin(BASE_URL, link_url)
        response = requests.get(full_url, headers=HEADERS, timeout=10)
        response.raise_for_status()

        if ".pdf" in full_url.lower():
            with fitz.open(stream=response.content, filetype="pdf") as doc:
                for page in doc:
                    text += page.get_text() + "\n"
        else:
            soup = BeautifulSoup(response.text, 'html.parser')
            content = soup.find('div', id='article')
            if not content:
                content = soup.find('body')
            
            if content:
                # 스크립트와 스타일 태그 제거
                for script_or_style in content(['script', 'style']):
                    script_or_style.decompose()
                text = content.get_text(separator="\n", strip=True)
            else:
                text = soup.get_text(separator="\n", strip=True)

    except Exception as e:
        print(f"  [오류] 링크에서 텍스트 추출 실패: {full_url} ({e})")
        
    time.sleep(0.5) 
    return clean_text(text)

def parse_historical_date(month_str, day_str, year):
    """ "January", "30-31", 2018을 datetime 객체로 변환 (회의 종료일 기준) """
    try:
        # "Jan." -> "Jan"
        month_key = month_str.strip('.')
        month = MONTH_MAP.get(month_key)
        
        if not month:
             # "Jan" 같은 축약형도 처리
            for k, v in MONTH_MAP.items():
                if k.startswith(month_key):
                    month = v
                    break
        if not month:
             raise ValueError(f"알 수 없는 월: {month_str}")

        # "30-31" -> 31, "1" -> 1
        day = int(re.split(r'[-/]', day_str)[-1]) 
        
        return datetime.datetime(year, month, day)
    except Exception as e:
         # print(f"  [경고] 날짜 파싱 실패: '{month_str} {day_str}' ({year}년) - {e}")
         return None

# --- 3. 메인 스크래핑 함수 (Historical) ---
def scrape_historical_minutes(output_file):
    print(f"--- FOMC 'Minutes' (의사록) 과거 데이터 스크래핑 시작 ---")
    print(f"대상 기간: {START_YEAR}년 ~ {END_YEAR}년")
    print(f"데이터 저장 위치: {output_file}\n")
    
    scraped_count = 0
    total_meetings = 0
    
    # --- [수정] 월(Month) 이름으로만 시작하는 행을 찾기 위한 정규식 생성 ---
    escaped_months = [re.escape(k) for k in MONTH_MAP.keys()]
    month_pattern = '|'.join(escaped_months)
    
    # [v3 수정] ^(시작) 앵커를 제거하고 \s* (공백)도 제거
    # (group 1): 월, \s+: 공백, (group 2): 날짜
    date_regex = re.compile(r'(' + month_pattern + r')\s+([\d-]+)')
    # --- [수정 완료] ---
    
    # START_YEAR부터 END_YEAR까지 역순으로 순회
    for year in tqdm(range(END_YEAR, START_YEAR - 1, -1), desc="연도별 수집"):
        
        historical_url = HISTORICAL_URL_TEMPLATE.format(year=year)
        
        soup = get_soup(historical_url)
        if not soup:
            print(f"  [경고] {year}년 페이지를 건너뜁니다 (접근 불가)")
            continue
            
        article = soup.find('div', id='article')
        if not article:
            print(f"  [경고] {year}년 페이지를 건너뜁니다 (article div 없음)")
            continue

        rows = article.find_all(['tr', 'p'])
        
        for row in rows:
            row_text = row.get_text(strip=True)

            # [v3 수정] .match() -> .search()
            # 문자열 '어디에서든' "월 + 날짜" 패턴을 찾음
            date_match = date_regex.search(row_text)
            
            if not date_match:
                # "pages 25", "Part 1" 등은 `month_pattern`에 없으므로 무시됨
                continue 
            
            month_str = date_match.group(1) # "January"
            day_str = date_match.group(2) # "30-31"
            
            meeting_date = parse_historical_date(month_str, day_str, year)
            if meeting_date is None:
                continue
                
            total_meetings += 1
            
            # 2. 'Minutes' 링크 찾기
            minutes_link = None
            links = row.find_all('a')
            for link in links:
                link_text = link.get_text(strip=True).lower()
                if 'minutes' in link_text:
                    minutes_link = link
                    break
            
            if minutes_link:
                link_url = minutes_link.get('href')
                
                if not link_url.startswith('http') and not link_url.startswith('/'):
                    if '.htm' in link_url and not link_url.startswith('/'):
                         link_url = f"/monetarypolicy/fomc/minutes/{link_url}"
                
                source_type = "FOMC Minutes (Historical)"
                text = extract_text_from_link(link_url)
                
                if text:
                    data = {
                        "date": meeting_date.strftime('%Y-%m-%d'), 
                        "year": year,
                        "source_type": source_type,
                        "url": urljoin(BASE_URL, link_url),
                        "text": text
                    }
                    save_to_jsonl(data, output_file)
                    scraped_count += 1
                else:
                    print(f"  [경고] {year} ({month_str} {day_str}) - 텍스트 추출 실패 (링크: {link_url})")
                        
    print(f"\n--- 스크래핑 완료 (2000-2019) ---")
    print(f"총 {total_meetings}개의 회의 항목 중 {scraped_count}개의 'Minutes' 문서를 수집했습니다.")

# --- 4. 메인 실행 ---
if __name__ == "__main__":
    if not os.path.exists(DOWNLOADS_DIR):
        os.makedirs(DOWNLOADS_DIR)
        print(f"'{DOWNLOADS_DIR}' 폴더를 생성했습니다.")

    if os.path.exists(OUTPUT_FILE):
        os.remove(OUTPUT_FILE)
        print(f"기존 파일 '{OUTPUT_FILE}'을 삭제했습니다. 새로 수집합니다.")
    
    scrape_historical_minutes(OUTPUT_FILE)
    
    print(f"\n--- 모든 'Minutes' 작업 완료 ---")
    print(f"최종 데이터가 '{OUTPUT_FILE}' 파일에 저장되었습니다.")


--- FOMC 'Minutes' (의사록) 과거 데이터 스크래핑 시작 ---
대상 기간: 2000년 ~ 2019년
데이터 저장 위치: C:\Users\82109\Downloads\statements\fomc_minutes_2000-2019.jsonl



연도별 수집: 100%|██████████| 20/20 [00:54<00:00,  2.75s/it]


--- 스크래핑 완료 (2000-2019) ---
총 137개의 회의 항목 중 43개의 'Minutes' 문서를 수집했습니다.

--- 모든 'Minutes' 작업 완료 ---
최종 데이터가 'C:\Users\82109\Downloads\statements\fomc_minutes_2000-2019.jsonl' 파일에 저장되었습니다.


In [12]:
# [디버깅 스크립트]
# 2000-2019년 과거 데이터 목록 페이지(2019년)의 HTML을 저장합니다.
# 0개가 수집되는 원인을 파악하기 위함입니다.

import requests
import os
import datetime

# --- 1. 설정 ---
# 2019년 페이지를 샘플로 저장
YEAR_TO_DEBUG = 2019
HISTORICAL_URL_TEMPLATE = "https://www.federalreserve.gov/monetarypolicy/fomchistorical{year}.htm"
DEBUG_URL = HISTORICAL_URL_TEMPLATE.format(year=YEAR_TO_DEBUG)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# --- 2. 저장 위치 ---
# (이 스크립트를 실행하는 폴더와 동일한 위치)
DOWNLOADS_DIR = os.path.join(os.path.expanduser('~'), 'Downloads', 'statements')
OUTPUT_FILE = os.path.join(DOWNLOADS_DIR, f"debug_historical_{YEAR_TO_DEBUG}.html")

# --- 3. 메인 실행 ---
def main():
    print(f"--- {YEAR_TO_DEBUG}년 '목록' 페이지 HTML 디버깅 시작 ---")
    print(f"대상 URL: {DEBUG_URL}")

    try:
        # 1. 폴더 생성
        if not os.path.exists(DOWNLOADS_DIR):
            os.makedirs(DOWNLOADS_DIR)

        # 2. 웹페이지 요청
        response = requests.get(DEBUG_URL, headers=HEADERS)
        response.raise_for_status() # 오류가 있으면 예외 발생
        
        # 3. HTML 텍스트 저장
        # (BeautifulSoup 파싱 안함! 원본 그대로 저장)
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            f.write(response.text)
            
        print(f"\n--- 성공! ---")
        print(f"HTML 원본 파일이 '{OUTPUT_FILE}' 경로에 저장되었습니다.")
        print("이 파일을 저에게 업로드해주세요.")

    except requests.exceptions.RequestException as e:
        print(f"\n[오류] 페이지에 접근할 수 없습니다: {e}")
    except IOError as e:
        print(f"\n[오류] 파일 저장에 실패했습니다: {e}")
    except Exception as e:
        print(f"\n[알 수 없는 오류] {e}")

if __name__ == "__main__":
    main()

--- 2019년 '목록' 페이지 HTML 디버깅 시작 ---
대상 URL: https://www.federalreserve.gov/monetarypolicy/fomchistorical2019.htm

--- 성공! ---
HTML 원본 파일이 'C:\Users\82109\Downloads\statements\debug_historical_2019.html' 경로에 저장되었습니다.
이 파일을 저에게 업로드해주세요.


In [14]:
# FOMC 'Minutes' (의사록) 전용 크롤러 (2000 ~ 2019년 과거 데이터)
#
# [2025-10-30 v6 수정]
# - 2007-2019년의 'div.panel' 구조 (debug_historical_2019.html 기반)
# - 2000-2006년의 'p/tr' 구조
#
# [v6 핵심 수정]
# - find_minutes_link 함수가 '링크 텍스트'(예: "HTML") 대신
#   'URL 주소(href)'에 'fomcminutes'가 포함되어 있는지 검사하도록 수정.
# - 2000-2006년 방식을 위해 'record of policy actions' 텍스트 검사 로직은 유지.
#
# [중요] 실행 전 설정:
# 1. 터미널에서 `pip install requests beautifulsoup4 PyMuPDF tqdm`을 실행하여
#    라이브러리를 설치하세요.

import requests
from bs4 import BeautifulSoup
import fitz  # PyMuPDF
import json
import time
import os
import datetime
from urllib.parse import urljoin
from tqdm import tqdm
import re

# --- 1. 설정 (Configuration) ---
BASE_URL = "https://www.federalreserve.gov"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
HISTORICAL_URL_TEMPLATE = "https://www.federalreserve.gov/monetarypolicy/fomchistorical{year}.htm"
DOWNLOADS_DIR = os.path.join(os.path.expanduser('~'), 'Downloads', 'statements')
OUTPUT_FILE = os.path.join(DOWNLOADS_DIR, "fomc_minutes_2000-2019.jsonl")
START_YEAR = 2000 
END_YEAR = 2019

MONTH_MAP = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12,
    'Jan.': 1, 'Feb.': 2, 'Mar.': 3, 'Apr.': 4, 'Jun.': 6, 'Jul.': 7, 'Aug.': 8, 'Sept.': 9, 'Oct.': 10, 'Nov.': 11, 'Dec.': 12,
    'April/May': 4
}

# --- 2. 헬퍼 함수 (Helper Functions) ---

def save_to_jsonl(data, outfile):
    """데이터를 JSON Lines (.jsonl) 파일에 추가합니다."""
    try:
        output_dir = os.path.dirname(outfile)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        with open(outfile, 'a', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False)
            f.write('\n')
    except IOError as e:
        print(f"  [오류] 파일 쓰기 실패: {e}")

def get_soup(url):
    """지정된 URL의 BeautifulSoup 객체를 반환합니다."""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        response.encoding = response.apparent_encoding 
        return BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"  [오류] URL에 접근할 수 없습니다: {url} ({e})")
        return None

def clean_text(text):
    """불필요한 공백과 줄바꿈 문자를 정제합니다."""
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def extract_text_from_link(link_url):
    """HTML 또는 PDF 링크에서 텍스트를 깨끗하게 추출합니다."""
    text = ""
    try:
        full_url = urljoin(BASE_URL, link_url)
        
        response = requests.get(full_url, headers=HEADERS, timeout=10)
        response.raise_for_status()

        if ".pdf" in full_url.lower():
            with fitz.open(stream=response.content, filetype="pdf") as doc:
                for page in doc:
                    text += page.get_text() + "\n"
        else:
            soup = BeautifulSoup(response.text, 'html.parser')
            # [사용자님 확인] <div id="article">이 본문이 맞습니다.
            content = soup.find('div', id='article') 
            if not content:
                content = soup.find('body')
            
            if content:
                for script_or_style in content(['script', 'style']):
                    script_or_style.decompose()
                text = content.get_text(separator="\n", strip=True)
            else:
                text = soup.get_text(separator="\n", strip=True)

    except Exception as e:
        print(f"  [오류] 링크에서 텍스트 추출 실패: {full_url} ({e})")
        
    time.sleep(0.5) 
    return clean_text(text)

def parse_date_from_text(text, year):
    """ 텍스트에서 '월'과 '일'을 찾아 datetime 객체로 변환 """
    try:
        escaped_months = [re.escape(k) for k in MONTH_MAP.keys()]
        month_pattern = '|'.join(escaped_months)
        date_regex = re.compile(r'(' + month_pattern + r')\s+([\d-]+)')
        
        date_match = date_regex.search(text)
        if not date_match:
            return None 

        month_str_key = date_match.group(1) # "April/May" 또는 "Jan."
        day_str = date_match.group(2) # "30-31"
        
        month = MONTH_MAP.get(month_str_key)
        
        if month_str_key == 'April/May':
            month = 4
            day = int(re.split(r'[-/]', day_str)[-1])
            if day == 1: 
                month = 5
        else:
            if not month:
                 # "Jan." -> "Jan"
                 month_str_key = month_str_key.strip('.')
                 month = MONTH_MAP.get(month_str_key)
                 
            if not month:
                 raise ValueError(f"알 수 없는 월: {date_match.group(1)}")
            day = int(re.split(r'[-/]', day_str)[-1]) 
        
        return datetime.datetime(year, month, day)

    except Exception as e:
         print(f"  [경고] 날짜 파싱 실패: '{text}' ({year}년) - {e}")
         return None

# --- [v6 수정] ---
def find_minutes_link(element):
    """ 주어진 HTML 요소(element) 안에서 'Minutes' 링크를 찾음 """
    links = element.find_all('a')
    for link in links:
        href = link.get('href', '')
        link_text = link.get_text(strip=True).lower()

        # [v6 수정] Priority 1: URL에 'fomcminutes'가 있는지 (2007-2019년 방식)
        # 예: /monetarypolicy/fomcminutes20190130.htm
        if 'fomcminutes' in href and 'transcript' not in href:
            return href
        
        # [v6 수정] Priority 2: 링크 텍스트에 "minutes" 또는 "record"가 있는지 (2000-2006년 방식)
        is_minutes = 'minutes' in link_text
        is_record = 'record of policy actions' in link_text
        is_not_transcript = 'transcript' not in link_text
        
        if (is_minutes or is_record) and is_not_transcript:
             return href
             
    return None # 링크 못 찾음

# --- 3. 메인 스크래핑 함수 (v5 로직 유지, v6 함수 적용) ---
def scrape_historical_minutes(output_file):
    print(f"--- FOMC 'Minutes' (의사록) 과거 데이터 스크래핑 시작 (v6) ---")
    print(f"대상 기간: {START_YEAR}년 ~ {END_YEAR}년")
    print(f"데이터 저장 위치: {output_file}\n")
    
    scraped_count = 0
    total_meetings = 0
    
    for year in tqdm(range(END_YEAR, START_YEAR - 1, -1), desc="연도별 수집"):
        
        historical_url = HISTORICAL_URL_TEMPLATE.format(year=year)
        soup = get_soup(historical_url)
        if not soup:
            print(f"  [경고] {year}년 페이지를 건너뜁니다 (접근 불가)")
            continue
            
        article = soup.find('div', id='article')
        if not article:
            print(f"  [경고] {year}년 페이지를 건너뜁니다 (article div 없음)")
            continue

        # --- 모드 1: 2007-2019년 방식 ('div.panel' 구조) ---
        meetings_panels = article.find_all('div', class_='panel-default')
        
        if meetings_panels:
            
            for panel in meetings_panels:
                # 1. 날짜 찾기 (h5 태그)
                h5 = panel.find('h5')
                if not h5: continue
                
                date_text = h5.get_text(strip=True) 
                meeting_date = parse_date_from_text(date_text, year)
                if meeting_date is None:
                    continue
                
                total_meetings += 1
                
                # 2. 'Minutes' 링크 찾기 (v6 함수 사용)
                link_url = find_minutes_link(panel)
                
                if link_url:
                    text = extract_text_from_link(link_url)
                    if text:
                        data = {
                            "date": meeting_date.strftime('%Y-%m-%d'), 
                            "year": year,
                            "source_type": "FOMC Minutes (Historical)",
                            "url": urljoin(BASE_URL, link_url),
                            "text": text
                        }
                        save_to_jsonl(data, output_file)
                        scraped_count += 1
        
        # --- 모드 2: 2000-2006년 방식 ('p' 또는 'tr' 구조) ---
        else:
            rows = article.find_all(['tr', 'p'])
            
            for row in rows:
                row_text = row.get_text(strip=True)
                
                # 1. 날짜 찾기
                meeting_date = parse_date_from_text(row_text, year)
                if meeting_date is None:
                    continue 
                
                total_meetings += 1
                
                # 2. 'Minutes' 링크 찾기 (v6 함수 사용)
                link_url = find_minutes_link(row)
                
                if link_url:
                    text = extract_text_from_link(link_url)
                    if text:
                        data = {
                            "date": meeting_date.strftime('%Y-%m-%d'), 
                            "year": year,
                            "source_type": "FOMC Minutes (Historical)",
                            "url": urljoin(BASE_URL, link_url),
                            "text": text
                        }
                        save_to_jsonl(data, output_file)
                        scraped_count += 1
                        
    print(f"\n--- 스크래핑 완료 (2000-2019) ---")
    print(f"총 {total_meetings}개의 회의 항목 중 {scraped_count}개의 'Minutes' 문서를 수집했습니다.")

# --- 4. 메인 실행 ---
if __name__ == "__main__":
    if not os.path.exists(DOWNLOADS_DIR):
        os.makedirs(DOWNLOADS_DIR)
        print(f"'{DOWNLOADS_DIR}' 폴더를 생성했습니다.")

    if os.path.exists(OUTPUT_FILE):
        os.remove(OUTPUT_FILE)
        print(f"기존 파일 '{OUTPUT_FILE}'을 삭제했습니다. 새로 수집합니다.")
    
    scrape_historical_minutes(OUTPUT_FILE)
    
    print(f"\n--- 모든 'Minutes' 작업 완료 ---")
    print(f"최종 데이터가 '{OUTPUT_FILE}' 파일에 저장되었습니다.")



기존 파일 'C:\Users\82109\Downloads\statements\fomc_minutes_2000-2019.jsonl'을 삭제했습니다. 새로 수집합니다.
--- FOMC 'Minutes' (의사록) 과거 데이터 스크래핑 시작 (v6) ---
대상 기간: 2000년 ~ 2019년
데이터 저장 위치: C:\Users\82109\Downloads\statements\fomc_minutes_2000-2019.jsonl



연도별 수집:   0%|          | 0/20 [00:00<?, ?it/s]

연도별 수집:  35%|███▌      | 7/20 [01:07<02:09,  9.93s/it]

  [경고] 날짜 파싱 실패: 'July 31-August 1  Meeting - 2012' (2012년) - invalid literal for int() with base 10: ''


연도별 수집: 100%|██████████| 20/20 [03:11<00:00,  9.58s/it]


--- 스크래핑 완료 (2000-2019) ---
총 185개의 회의 항목 중 168개의 'Minutes' 문서를 수집했습니다.

--- 모든 'Minutes' 작업 완료 ---
최종 데이터가 'C:\Users\82109\Downloads\statements\fomc_minutes_2000-2019.jsonl' 파일에 저장되었습니다.
